In [1]:
import sys
import os
from groq import Groq
sys.path.append(os.path.dirname(os.path.abspath("./")))
from src.tools import model_tools
from src.tools.base import Tool
from configs.config import GROQ_API_KEY
from dotenv import load_dotenv
from warnings import filterwarnings

filterwarnings("ignore")

Loaded env variables: False


In [11]:
load_dotenv("../secrets/environments/local.env")

True

In [2]:
# get tools
tool_list = []
for tool in model_tools.__all__:
    tool_list.append(Tool.from_function(function=getattr(model_tools, tool)))

openai_tools = [tool.to_openai_tool() for tool in tool_list]

In [3]:
openai_tools

[{'type': 'function',
  'function': {'name': 'multiply',
   'description': 'multiply(a: int, b: int) -> int\nMultiplies two integers and returns the result integer.\n\n    Args:\n        a (int): The first integer.\n        b (int): The second integer.\n\n    Returns:\n        int: The product of a and b.\n    ',
   'parameters': {'properties': {'a': {'title': 'A', 'type': 'integer'},
     'b': {'title': 'B', 'type': 'integer'}},
    'required': ['a', 'b'],
    'type': 'object'}}},
 {'type': 'function',
  'function': {'name': 'add',
   'description': 'add(a: int, b: int) -> int\nAdd two integers and returns the result integer.\n\n    Args:\n        a (int): The first integer.\n        b (int): The second integer.\n\n    Returns:\n        int: The sum of a and b\n    ',
   'parameters': {'properties': {'a': {'title': 'A', 'type': 'integer'},
     'b': {'title': 'B', 'type': 'integer'}},
    'required': ['a', 'b'],
    'type': 'object'}}},
 {'type': 'function',
  'function': {'name': 'su

In [14]:
model_service = Groq(timeout=120.0)

In [32]:
json_mode = False
response = model_service.chat.completions.create(
    model="llama-3.1-70b-versatile",
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": "What is 1 + 1?"},
        {
            "role": "assistant",
            "content": "",
            "tool_calls": [
                {
                    "id": "call_dhqx",
                    "type": "function",
                    "function": {"name": "add", "arguments": '{"a": 1, "b": 1}'},
                }
            ],
        },
        {"tool_call_id": "call_dhqx", "role": "tool", "name": "add", "content": "2"},
    ],
    tools=openai_tools,
    tool_choice="auto",
    response_format={"type": "json_object"} if json_mode else None,
    stop=None,
)

In [33]:
response.to_dict()

{'id': 'chatcmpl-39ec4a46-5604-43ae-b9ad-ba9ee37423f3',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': 'The answer to 1 + 1 is 2.', 'role': 'assistant'}}],
 'created': 1726585486,
 'model': 'llama-3.1-70b-versatile',
 'object': 'chat.completion',
 'system_fingerprint': 'fp_5c5d1b5cfb',
 'usage': {'completion_tokens': 13,
  'prompt_tokens': 916,
  'total_tokens': 929,
  'completion_time': 0.052217544,
  'prompt_time': 0.275113006,
  'queue_time': 0.13091876800000002,
  'total_time': 0.32733055},
 'x_groq': {'id': 'req_01j8087emvennsc30yz6pxpstv'}}